# Optimal Power Flow via PandaModels
### PandaModels.jl: Interfacing PowerModels with pandapower

This tutorial describes how to run the Optimal Power Flow via [PandaModels.jl](https://e2niee.github.io/PandaModels.jl/dev/) calling [PowerModels.jl](https://lanl-ansi.github.io/PowerModels.jl/stable/) package.

### Let's get started

So here is an example of how it works. First, we create a grid in pandapower. Here, we create a meshed 110kV grid with four buses that is fed from an 220kV network through a 3-Winding transformer.
Also we need to set min/max values as optimization constarints.

In [ ]:
from pandapower.create import (
    create_empty_network,
    create_bus,
    create_transformer3w_from_parameters,
    create_line,
    create_gen,
    create_load,
    create_poly_cost
)
from pandapower.runpm import runpm_ac_opf, runpm_dc_opf
from pandapower.run import runopp

In [ ]:
net = create_empty_network()

min_vm_pu = 0.95
max_vm_pu = 1.05

#create buses
bus1 = create_bus(net, vn_kv=220., geodata=(5,9), min_vm_pu=min_vm_pu, max_vm_pu=max_vm_pu)
bus2 = create_bus(net, vn_kv=110., geodata=(6,10), min_vm_pu=min_vm_pu, max_vm_pu=max_vm_pu)
bus3 = create_bus(net, vn_kv=110., geodata=(10,9), min_vm_pu=min_vm_pu, max_vm_pu=max_vm_pu)
bus4 = create_bus(net, vn_kv=110., geodata=(8,8), min_vm_pu=min_vm_pu, max_vm_pu=max_vm_pu)
bus5 = create_bus(net, vn_kv=110., geodata=(6,8), min_vm_pu=min_vm_pu, max_vm_pu=max_vm_pu)

#create 220/110/110 kV 3W-transformer
create_transformer3w_from_parameters(net, bus1, bus2, bus5, vn_hv_kv=220, vn_mv_kv=110,
                                        vn_lv_kv=110, vk_hv_percent=10., vk_mv_percent=10.,
                                        vk_lv_percent=10., vkr_hv_percent=0.5,
                                        vkr_mv_percent=0.5, vkr_lv_percent=0.5, pfe_kw=10,
                                        i0_percent=0.1, shift_mv_degree=0, shift_lv_degree=0,
                                        sn_hv_mva=100, sn_mv_mva=50, sn_lv_mva=50)

#create 110 kV lines
l1 = create_line(net, bus2, bus3, length_km=70., std_type='149-AL1/24-ST1A 110.0')
l2 = create_line(net, bus3, bus4, length_km=50., std_type='149-AL1/24-ST1A 110.0')
l3 = create_line(net, bus4, bus2, length_km=40., std_type='149-AL1/24-ST1A 110.0')
l4 = create_line(net, bus4, bus5, length_km=30., std_type='149-AL1/24-ST1A 110.0')

#create loads
create_load(net, bus2, p_mw=60)
create_load(net, bus3, p_mw=70)
create_load(net, bus4, p_mw=10)

#create generators
g1 = create_gen(net, bus1, p_mw=40, min_p_mw=0, max_p_mw=200, vm_pu=1.01, slack=True)
create_poly_cost(net, g1, 'gen', cp1_eur_per_mw=1)

g2 = create_gen(net, bus3, p_mw=40, min_p_mw=0, max_p_mw=200, vm_pu=1.01)
create_poly_cost(net, g2, 'gen', cp1_eur_per_mw=3)

g3 = create_gen(net, bus4, p_mw=50, min_p_mw=0, max_p_mw=200, vm_pu=1.01)
create_poly_cost(net, g3, 'gen', cp1_eur_per_mw=3)
net

Note that PowerModels does not have a 3W-transformer model, but since pandapower includes the equations to calculate the equivalent branches for the 3W-transformers (2W-transformers instead), it is possible to optimize grids with 3W-transformers in PowerModels through the pandapower interface. The same is true for other complex transformer models, switches/breaker, extended ward equivalents etc.

Let's have a look at the grid we created with pandapowers plotting module:

In [ ]:
import geojson
geojson.loads(net.bus.geo.iloc[0])["coordinates"]

In [ ]:
import pandapower.plotting as plot
plot.set_line_geodata_from_bus_geodata(net)
plot.simple_plot(net)

Now let's run an OPF through PowerModels and look at the results (Note that the first time the runpm function is called, Julia is started in the background, which may take some time):

In [ ]:
try:
    runpm_ac_opf(net)
except Exception as err:
    print(err)
# or: runpm(net, pm_model="ACPPowerModel")

Also, there are further parameters and options that you can add as input while calling the Optimization Problem from PandaModles:

| parameter | description | type | default |
| :--- | :--- | :---: | :--- |
| correct_pm_network_data | checks if network data is correct. If not tries to correct it | bool | True |
| silence | Suppresses information and warning messages output by PowerModels | bool | True |
| pm_model | PowerModels.jl model to use | str | "ACPPowerModel" |
| pm_solver | "main" solver| str | "ipopt" |
| pm_mip_solver | mixed integer solver| str | "cbc" |
| pm_nl_solver | nonlinear solver| str | "ipopt" |
| pm_tol | default desired convergence tolerance for solver to use | float | 1e-8 |
| pm_log_level | solver log level in power models | int | 0 |
| delete_buffer_file | If True, the .json file used by PandaModels will be deleted after optimization. | bool | True |


Since Generator 1 has the lowest cost, all required power is supplied through this generator:

In [ ]:
net.res_gen

This however leeds to an overload in the three-winding transformer, through which g1 is connected:

In [ ]:
net.res_trafo3w.loading_percent

Let's set some constraints for the 3W-transformer and the lines and rerun the OPF:

In [ ]:
net.trafo3w["max_loading_percent"] = 50
net.line["max_loading_percent"] = 20
try:
    runpm_ac_opf(net)
except:
    print("Cannot be performed due to [WinError 3] - Can't find file python39.dll")

The constraints are complied with for all lines and the 3W transformer:

In [ ]:
net.res_trafo3w.loading_percent

In [ ]:
net.res_line.loading_percent

The power is now generated by a mixture of the generators:

In [ ]:
net.res_gen

## Accessing the full functionality of PowerModels.jl

Apart from the AC OPF used in the example above, pandapower also has an interface to run the DC OPF:

In [ ]:
try:
    runpm_dc_opf(net)
except:
    print("Cannot be performed due to [WinError 3] - Can't find file python39.dll")
# or: runpm(net, pm_model="DCPPowerModel")
net.res_bus


The PowerModels data structure that was passed to Julia can be accessed like this:

In [ ]:
net._pm["bus"]

## Timings

Comparing the runopp function (that runs an OPF through PYPOWER) and the runpm function shows that PandaModels.jl is much more performant:

In [ ]:
%timeit runopp(net)

In [ ]:
try:
    %timeit runpm_ac_opf(net)
except:
    print("Cannot be performed due to [WinError 3] - Can't find file python39.dll")